# Tableau - Extract tables and queries

### 1. Set-up Notebook

In [1]:
from json import load
import pandas

pandas.set_option('display.max_rows', 1000)

In [2]:
#Loading Andrea's parse_sql_files script
%run ../sql_parse/parse_sql_files

In [3]:
def json_to_df(json_filepath):
    
    """
    Expects filepath to JSON file from extract_all function.
    Takes all tables and queries from Tableau extraction, passes queries through sql parsing script to find table names and columns
    Returns pandas dataframe containing columns [workbook_name, connection_source (keep?), table name, column]
    """
    
    lst = []
    
    with open(json_filepath) as f:
        all_sources = load(f)
    
        #loop through each workbook and it's attached datasources
        for workbook_name, datasources in all_sources.items():
            
            #Checking that there is atleast one datasource (i.e. dictionary is not empty)
            if bool(datasources):
                #Loop through each datasource name and its contents (e.g. table or query)
                for datasource_name, value in datasources.items():

                    #Grab connection type from datasource name.  Should be table or query
                    connection_type = datasource_name.split('-')[0]

                    if connection_type == 'table':
                        #For direct table connections, reports empty column value.  Tableau basically imports all columns from a direct table connection, but more difficult to tell which ones are actually used in the visualization
                        column = ''
                        lst.append([workbook_name, value, column])

                    elif connection_type == 'query':
                        #Parse SQL query 
                        psf = parse_sql_files(value)
                        table_columns = psf.get_basic_tables_and_columns()['tables']

                        #If parse_sql_files struggles to find any table names, converts table to empty string so workbook is still added to list and included in dataframe
                        for table, columns in table_columns.items():
                            #If the sql parsing is able to identify columns, it adds a row for each table-column pairing
                            if len(columns)>0:
                                for column in columns:
                                    column = column.split('.')[1]
                                    lst.append([workbook_name, table, column])
                            #If the sql parsing cannot identify columns, returns a single row with empty column
                            elif len(columns)==0:
                                column = ''
                                lst.append([workbook_name, table, column])
            else:
                lst.append([workbook_name, '', ''])
                        
    df = pandas.DataFrame(lst, columns=['workbook_name', 'table', 'column'])
    
    return df

In [4]:
df = json_to_df('data/all_sources.json')

In [5]:
df.to_csv('data/tableau_extractions.csv')